In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pickle

# 카카오페이지에서 데이터 크롤링

In [14]:
# 카카오페이지 로그인 후 쿠키 저장하는 파트

url = 'https://page.kakao.com'
options = Options()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
data_list = []

driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)

driver.execute_script(f"document.querySelector('button.pr-16pxr').click();")
time.sleep(3)

id_input = driver.find_element(By.CSS_SELECTOR, '#loginId--1')
id_input.send_keys('tkdehdjajsl@gmail.com')
pw_input = driver.find_element(By.CSS_SELECTOR, '#password--2')
pw_input.send_keys('gksruf9507')
time.sleep(5)

submit_btn = driver.find_element(By.CSS_SELECTOR, 'button.submit')
submit_btn.click()
time.sleep(5)

# 로그인 후 쿠키 저장
cookies = driver.get_cookies()
with open('kakao_cookies.pkl', 'wb') as f:
  pickle.dump(cookies, f)

driver.quit()

In [19]:
# 상세 페이지에서 데이터 가져오는 파트

url = 'https://page.kakao.com'
options = Options()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
data_list = []

# 여기에 로그인 파트

driver = webdriver.Chrome(options=options)

driver.get(url)

# 쿠키 로드
with open('kakao_cookies.pkl', 'rb') as f:
  cookies = pickle.load(f)
  for cookie in cookies:
    driver.add_cookie(cookie)

driver.refresh()

time.sleep(3)

df = pd.read_csv('KakaopageLinks.csv')
count = 0
for url in df['link'].iloc[:10001]:
  data = {}
  count += 1

  try:
    # 두번째 페이지 먼저 호출
    driver.get(url+'?tab_type=about')
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
      
    # JavaScript를 사용하여 버튼이 존재하면 클릭
    script = f"""
    var button = document.querySelector('button.font-medium1.flex.h-59pxr.cursor-pointer.items-center.justify-center.border-0.border-t-1.border-solid.border-line-10.px-24pxr.text-el-50');
    if (button) {{
        button.click();
    }}
    """
    driver.execute_script(script)

    title = soup.select_one('span.font-large3-bold.mb-3pxr.text-ellipsis.break-all.text-el-70.line-clamp-2').text
    if '단행본' in title:
      continue
    else:
      data['title'] = title

    synopsis = soup.select_one('span.font-small1.mb-8pxr.block.whitespace-pre-wrap.break-words.text-el-70').get_text()
    data['synopsis'] = synopsis

    author = soup.select_one('span.font-small2.mb-6pxr.text-ellipsis.text-el-70.opacity-70.break-word-anywhere.line-clamp-2').get_text()
    data['author'] = author
    genre = soup.select('div.text-ellipsis.line-clamp-1.text-el-70.opacity-70 > span')[1].get_text()
    data['genre'] = genre

    # 첫번째 페이지 호출
    driver.get(url)

    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    score = soup.find('img', {'alt': '별점'}).find_next_sibling('span').get_text()
    data['score'] = score
    completion = soup.select_one('div.mt-6pxr.flex.items-center > span').get_text()
    data['completion'] = completion

    review_count = soup.select_one('div.flex.h-44pxr.w-full.flex-row.items-center.justify-between.px-18pxr.bg-bg-a-20 > div > span').get_text()
    data['review_count'] = review_count

    reviews = soup.select('span.font-medium2.whitespace-pre-wrap.text-el-70.break-word-anywhere')
    data['reviews'] = [review.get_text().strip() for review in reviews]
    
    data_list.append(data)

    if count % 100 == 0:
      print(count)
  except Exception as e:
    continue


df1 = pd.DataFrame(data_list)
df1.to_csv('KakaoData01.csv', index=False, encoding='utf-8-sig')

driver.quit()

Exception ignored in: <function Service.__del__ at 0x00000281BCEB5000>
Traceback (most recent call last):
  File "c:\KDT-HANG\M5_머신러닝\2차 프로젝트\team3\lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\KDT-HANG\M5_머신러닝\2차 프로젝트\team3\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\KDT-HANG\M5_머신러닝\2차 프로젝트\team3\lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\urllib\request.py", line 536, in _open
    result =

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.90)
Stacktrace:
	GetHandleVerifier [0x00007FF6EE349632+30946]
	(No symbol) [0x00007FF6EE2FE3C9]
	(No symbol) [0x00007FF6EE1F6FDA]
	(No symbol) [0x00007FF6EE1CCB85]
	(No symbol) [0x00007FF6EE2737A7]
	(No symbol) [0x00007FF6EE28A771]
	(No symbol) [0x00007FF6EE26C813]
	(No symbol) [0x00007FF6EE23A6E5]
	(No symbol) [0x00007FF6EE23B021]
	GetHandleVerifier [0x00007FF6EE47F83D+1301229]
	GetHandleVerifier [0x00007FF6EE48BDB7+1351783]
	GetHandleVerifier [0x00007FF6EE482A03+1313971]
	GetHandleVerifier [0x00007FF6EE37DD06+245686]
	(No symbol) [0x00007FF6EE30758F]
	(No symbol) [0x00007FF6EE303804]
	(No symbol) [0x00007FF6EE303992]
	(No symbol) [0x00007FF6EE2FA3EF]
	BaseThreadInitThunk [0x00007FFAB00E257D+29]
	RtlUserThreadStart [0x00007FFAB0C4AF28+40]


In [18]:
df = pd.read_csv('KakaoData01.csv')

df

,title,synopsis,author,genre,score,completion,review_count,reviews
0,내 머릿속 소드마스터,[왕자를 죽이고 싶어요!]\n\n이제부터 내 꿈은 프린스 슬레이어다.,괴도고구마,판타지,2.5,완결,전체 152,[]
1,퇴장한 악녀의 브런치 카페,죽고 눈떠 보니 빙의했더라.\n이런 소설 같은 이야기가 내게도 일어났다.\n연애 한...,지미신,로판,9.8,월~금 연재,전체 115,['나나리양이 오라버니 본인인 것 같은데요...그래서 답장을 못하시는 것 같은데.....
2,이무기전,“황녀 중 하나를 환국 태자와 혼인시킬 것이다.”\n\n려국을 패망의 길로 걷게 할...,연그늘,로맨스,9.8,완결,전체 73,"['안녕하세요, 연그늘입니다.\n새로운 작품으로 오랜만에 인사드려요.\n억겁의 세월..."


# 네이버 시리즈에서 데이터 크롤링

- 로맨스 페이지 : 1740 / 코드 201
- 로판 페이지 : 561 / 코드 207
- 판타지 페이지 : 466 / 코드 202
- 현판 페이지 : 372 / 코드 208
- 무협 페이지 : 210 / 코드 206

In [8]:
# 네이버 시리즈 카테고리별 링크 추출

base_url = 'https://series.naver.com'
url = 'https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=201&orderTypeCode=sale&is&isFinished=false&page={}'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data = []
for i in range(1,1741):
  response = requests.get(url.format(i), headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  lst_list = soup.select('ul.lst_list > li')
  for lst in lst_list:
    link = lst.select_one('div > h3 > a')['href']
    data.append(base_url+link)

df = pd.DataFrame({'link':data})
df.to_csv('genre_201.csv', index=False)

In [2]:
import pandas as pd

df = pd.read_csv('SeriesLinks.csv')

In [9]:
url = df['link'][0]

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

synopsis = soup.select('#content > div.end_dsc > div')

print(synopsis)

<br/>환관들의 은밀한 세상에 한 여인이 뛰.. 금녀(禁女)의 구역. 
					
<span class="al_r"><a class="lk_more _toggleMore(synopsis)" href="#" onclick="nclk_v2(event, 'end.storemore', '1200383');">더보기<em></em></a></span>
</div>, <div class="_synopsis" style="display: none">
<br/>그녀의 고민 상담에 구중궁궐이 들썩이기 시작한다.없는 환관까지 되었으니……. 놈이는 사실 남장여인이다. 
						<span class="al_r"><a class="lk_more up _toggleMore(synopsis)" href="#" onclick="nclk_v2(event, 'end.storeclose', '1200383');">접기<em></em></a></span>
</br></br></br></div>]


In [3]:
options = Options()
options.add_argument("--headless")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

datas = []

df = pd.read_csv('SeriesLinks.csv')
count = 0
for link in df['link'].iloc[20001:40001]:
  count += 1
  driver.get(link)
  time.sleep(1)

  html = driver.page_source

  soup = BeautifulSoup(html, 'html.parser')

  data = {}
  try:
    try:
      synopsis = soup.select('div.end_dsc > div')[1].get_text().strip()
    except:
      synopsis = soup.select_one('div.end_dsc > div._synopsis').get_text().strip()
    title = soup.select_one('#content > div.end_head > h2').get_text().strip()
    score = soup.select_one('div.score_area > em').get_text().strip()
    review_count = soup.select_one('span.u_cbox_count').get_text().strip()
    review_list = soup.select('span.u_cbox_contents')
    completed = soup.select_one('li.info_lst > ul > li > span').get_text().strip()
    genre = soup.select_one('li.info_lst > ul > li > span > a').get_text().strip()
    author = soup.select_one('li.info_lst > ul > li > a').get_text().strip()

    data['title'] = title
    data['genre'] = genre
    data['author'] = author
    data['score'] = score
    data['completed'] = completed
    data['review_count'] = review_count
    data['synopsis'] = synopsis
    data['reviews'] = [review.get_text().strip() for review in review_list]

    if count % 100 == 0:
      print(count)
    
  except Exception as e:
    continue

  datas.append(data)

driver.quit()

df1 = pd.DataFrame(datas)
df1.to_csv('SeriesData.csv', index=False, encoding='utf-8-sig')

100
600
700
1300
1400
1600
1700
1800
1900
2100
2200
2500
2700
2800
2900
3100
3400
3500
3800
4200
4300
4700
4900
5200
5900
6200
6500
6600
6700
6900
7000
7200
7600
7700
7800
8000
8200
8300
8600
8800
8900
9000
9100
9300
9500
9700
9800
9900
10400
10500
11300
11400
11600
12100
12200
12300
12500
12600
12700
12900
13000
13100
13300
13400
13900
14000
14700
15000
15300
15400
15500
15600
15900
16000
16100
16300
16800
16900
17000
17100
17400
17700
17800
17900
18000
18200
18600
18800
19300
19400
19600
19700
19800


In [ ]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7])

df.to_csv('SeriesData.csv', index=False, encoding='utf-8-sig')